### Build the Neural Network
- 신경망은 데이터에 대한 연산을 수행하는 `계층(layer)` / `모듈(module)`로 구성
- `torch.nn`: 신경망을 구성하는데 필요한 모든 구성 요소를 제공
- PyTorch의 모든 모듈은 `nn.Module`의 하위 클래스(subclass)
- 신경망은 다른 모듈(계층, layer)로 구성된 모듈

FashionMNIST dataset의 이미지를 분류하는 신경망

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda" if torch.cuda.is_available() else "cpu"
)
print(f"Using {device} device")

Using cuda device


#### 클래스 정의하기
- 신경망 모델을 `nn.Module`의 하위클래스로 정의하고, `__init__`에서 신경망 계층들을 초기화
- `nn.Module`을 상속받은 모든 클래스는 `forward` 메소드에 입력 데이터에 대한 연산을 구현

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        """
        nn.module.__init__()도 동작 가능
        -> 부모 클래스의 초기화 함수를 직접 호출하기 때문에, 부모 클래스가 바뀌면 코드가 망가짐
        """
        self.flatten = nn.Flatten()     # 28x28 이미지를 1차원 벡터 (784,)로 flatten
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

`NeuralNetwork`의 인스턴스(instance)를 생성하고 이를 `device`로 이동한 뒤, 구조(structure)를 출력

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)        # 가짜 이미지
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9], device='cuda:0')


#### 모델 계층 (Layer)

In [6]:
# 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 확인
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features = 28*28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 2.4109e-01, -1.6218e-01, -7.2167e-02,  8.3553e-02,  3.4714e-01,
         -2.5297e-01, -3.5880e-02, -8.4441e-03, -6.2864e-01, -1.3338e-01,
         -3.8685e-01,  1.8297e-02, -4.5151e-01, -7.5894e-02, -3.3265e-01,
          1.9389e-01,  6.9175e-03, -4.3012e-01, -1.4229e-01,  1.0442e-01],
        [ 1.3508e-04,  3.6364e-01, -1.3377e-01,  3.3162e-01,  2.7077e-01,
          1.4249e-01,  2.1699e-01, -3.1769e-01, -5.4999e-01, -2.2228e-01,
         -3.9262e-01, -1.9205e-01,  1.2123e-01, -4.5552e-01, -2.0670e-01,
          8.3712e-03, -3.7793e-01, -1.3921e-01, -2.3048e-01,  1.9537e-01],
        [ 1.8829e-01, -6.6159e-02, -2.7765e-01,  5.0809e-02, -1.4006e-01,
         -9.5918e-02,  3.7231e-02,  3.1169e-01, -5.1646e-01, -2.7615e-01,
         -4.3791e-01, -5.2397e-01, -7.2049e-03, -4.4123e-01, -8.2962e-02,
          3.0614e-01, -3.1465e-01, -1.1763e-01, -2.3581e-01,  5.6692e-02]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[2.4109e-01, 0.0000e+00, 0.0000e+00, 8.3

In [10]:
seq_model = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_model(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")


Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0355,  0.0081, -0.0311,  ...,  0.0138, -0.0323, -0.0340],
        [-0.0096, -0.0074, -0.0017,  ...,  0.0102, -0.0336, -0.0249]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0081, -0.0343], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0267,  0.0077,  0.0006,  ...,  0.0065,  0.0383, -0.0331],
        [ 0.0092, -0.0192,  0.0312,  ...,  0.0150,  0.0335, -0.0108]],
       device='cuda:0', grad_fn=<Sl